here original data is used to truncate the lists of time series columns and unlike previous attempts, failure columns are kept to be used as categorical columns.

In [3]:
import pandas as pd

In [12]:
original_data = pd.read_parquet("../../data/test_107_reduced.parquet")
original_data

,doorcyclecounter,doorforce,doorspeed,doorposition,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,"[-75.1, -75.1, -92.8, -73.5, -51.9, -46.3, -46...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0531, 0.078...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0451, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,"[-75.1, -75.1, -85.0, -92.1, -52.8, -51.6, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.033, 0.0822...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.048, -...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,"[-75.1, -75.1, -87.9, -88.3, -61.2, -51.1, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0612, 0.086...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0468, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,"[-75.1, -75.1, -85.6, -83.4, -57.2, -51.0, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0505, 0.087...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0474, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,"[-75.1, -75.1, -78.5, -91.4, -59.5, -49.7, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0297, 0.073...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0483, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,"[-75.1, -75.1, -93.3, -59.5, -38.7, -29.9, -29...","[0.0016, 0.0016, 0.0016, 0.0016, 0.054, 0.0706...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0385, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,120.000000,0.87,0.9
4876,6658,"[-75.1, -75.1, -90.3, -87.9, -55.8, -47.9, -47...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0462, 0.078...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0401, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,111.699997,0.86,1.0
4877,6659,"[-75.1, -75.1, -90.5, -78.1, -52.6, -49.0, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0475, 0.081...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,120.900002,0.89,1.0
4878,6660,"[-75.1, -75.1, -89.8, -82.6, -53.0, -49.3, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0543, 0.082...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,111.500000,0.86,0.8


Defining relevant column types

In [14]:
failure_columns = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

environment_columns = [
	'cmcouplerfriction',
	'cmdoorfriction',
	'cmelectronicage',
	'cmvibration',
]

static_state_columns = [
	'doorcyclecounter',
]

time_state_columns = [
	'doorforce',
	'doorspeed',
	'doorposition',
	# 'time',
]

In [15]:
doorforce_series = original_data['doorforce'].apply(pd.Series)
doorspeed_series = original_data['doorspeed'].apply(pd.Series)
doorposition_series = original_data['doorposition'].apply(pd.Series)

doorforce_series = doorforce_series.add_prefix('doorforce_')
doorspeed_series = doorspeed_series.add_prefix('doorspeed_')
doorposition_series = doorposition_series.add_prefix('doorposition_')
doorforce_series


,doorforce_0,doorforce_1,doorforce_2,doorforce_3,doorforce_4,doorforce_5,doorforce_6,doorforce_7,doorforce_8,doorforce_9,...,doorforce_447,doorforce_448,doorforce_449,doorforce_450,doorforce_451,doorforce_452,doorforce_453,doorforce_454,doorforce_455,doorforce_456
0,-75.099998,-75.099998,-92.800003,-73.500000,-51.900002,-46.299999,-46.299999,-38.299999,-48.000000,-21.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-75.099998,-75.099998,-85.000000,-92.099998,-52.799999,-51.599998,-51.599998,-50.000000,-36.700001,-44.200001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-75.099998,-75.099998,-87.900002,-88.300003,-61.200001,-51.099998,-51.099998,-50.200001,-49.099998,-31.700001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-75.099998,-75.099998,-85.599998,-83.400002,-57.200001,-51.000000,-51.000000,-50.400002,-33.099998,-35.700001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-75.099998,-75.099998,-78.500000,-91.400002,-59.500000,-49.700001,-49.700001,-49.200001,-37.799999,-46.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,-75.099998,-75.099998,-93.300003,-59.500000,-38.700001,-29.900000,-29.900000,-37.500000,-36.400002,-23.700001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4876,-75.099998,-75.099998,-90.300003,-87.900002,-55.799999,-47.900002,-47.900002,-41.200001,-38.299999,-32.299999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4877,-75.099998,-75.099998,-90.500000,-78.099998,-52.599998,-49.000000,-49.000000,-42.000000,-40.700001,-29.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4878,-75.099998,-75.099998,-89.800003,-82.599998,-53.000000,-49.299999,-49.299999,-41.900002,-41.500000,-28.200001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:

truncated_data = original_data.copy()
truncated_data.drop(columns=['doorforce', 'doorspeed', 'doorposition'], inplace=True)
truncated_data


,doorcyclecounter,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,120.000000,0.87,0.9
4876,6658,0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,111.699997,0.86,1.0
4877,6659,0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,120.900002,0.89,1.0
4878,6660,0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,111.500000,0.86,0.8


In [17]:
train_data = pd.concat([truncated_data, doorforce_series, doorspeed_series, doorposition_series], axis=1)
train_data

,doorcyclecounter,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,...,doorposition_447,doorposition_448,doorposition_449,doorposition_450,doorposition_451,doorposition_452,doorposition_453,doorposition_454,doorposition_455,doorposition_456
0,1166,0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1167,0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1168,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1169,0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1170,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4876,6658,0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4877,6659,0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4878,6660,0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Organizing the data based on failure types

In [20]:
train_data.to_parquet("../../data/preprocessed_tabular_truncated.parquet")
